# Local Search for the TSP

Consider the classical Traveling Salesman Problem (TSP, see the Classical Problems Library). The class provided below represents the TSP. Objects of this class represent instances of the TSP.

The data of the instances is provided in the files contained in the `tsp_instances` folder. For each instance we are given the data of the problem (files `*.tsp`) and the optimal solution, if exists (files`*.opt.tour`). The package `tsplib95` reads such files and makes its data available in a suitable Python object. The package is described here https://pypi.org/project/tsplib95/.

The data of the instances is taken from a set of known benchmark instances (known as the TSPLIB 95 library)
which are made available at this link http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/.
In the TSPLIB there are several instances of various sizes. For several instances the library provides also the
optimal solution, if it is known (some instances are so difficult that the optimal solution has not yet been found). In our example we will use a small instance for which we know the optimal solution.

- Q1. Find solutions to the given instance of the TSP using a local search algorithm
- Q2. Compare the solution you obtain to the optimal solution (the optimal solution and its cost are given by the class below). 

In [2]:
%pip install wrapt
import tsplib95


class TSP:

    def __init__(self,instance_name:str, optimal_tour:str):
        '''
        Generates the data of the instance given the name of the file
        containing the data of the instance and the name of the file containing the optimal solution.
        '''
        self.data = tsplib95.load('tsp_instances/'+ instance_name)
        self.optimal_tour = tsplib95.load('tsp_instances/' + optimal_tour)

    def get_nodes(self):
        '''
        Returns the list of nodes.
        '''
        return list(self.data.get_nodes())

    def get_n_nodes(self):
        return len(self.get_nodes())

    def get_distances(self):
        '''
        Returns the distances matrix.
        '''
        return self.data.edge_weights

    def get_distance(self,i:int,j:int):
        '''
        Returns the distance between given nodes i and j.
        '''
        return self.data.get_weight(i,j)

    def get_node_coordinates(self,i:int):
        '''
        Returns the coordinates of the nodes.
        '''
        print(self.data.node_coords)

    def get_n_optimal_tours(self):
        '''
        Returns the list of optimal tours (there may be more than one, though often only one).
        '''
        return len(self.optimal_tour.tours)

    def get_optimal_tour(self,n_tour=0):
        '''
        Returns a specific optimal tour (with no arguments returns the first).
        '''
        return self.optimal_tour.tours[n_tour]

    def get_optimal_tour_length(self,n_tour=0):
        '''
        Returns the length (objective value) of an optimal tour (default the first tour).
        '''
        return self.get_tour_length(self.optimal_tour.tours[n_tour])

    def get_tour_length(self, tour:list):
        '''
        Calculates the length (objective value) of a tour we pass as an argument.
        Note that the tour is first added to a list since the method trace_tours accepts only lists of tours.
        '''
        tour_list = [tour]
        return self.data.trace_tours(tour_list)[0]


Note: you may need to restart the kernel to use updated packages.


We create a small TSP instance

In [3]:
tsp = TSP("bays29.tsp","bays29.opt.tour")

In [4]:
tsp.get_optimal_tour()

[1,
 28,
 6,
 12,
 9,
 5,
 26,
 29,
 3,
 2,
 20,
 10,
 4,
 15,
 18,
 17,
 14,
 22,
 11,
 19,
 25,
 7,
 23,
 27,
 8,
 24,
 16,
 13,
 21]

In [5]:
tsp.get_optimal_tour_length()

2020

# Solution

We create a class that implements the LS algorithm

In [6]:
import copy
import random

class LocalSearch:

    def __init__(self,tsp:TSP):
        self.tsp = tsp

    def generate_inital_solution(self):
        '''
        Does the following:
        (i) Gets the list of nodes of the tsp problem
        (ii) Makes a deep copy of the list of nodes.
        This ensures that by changing the order in the initial solution we
        do not modify the list of nodes in the tsp class.
        (iii) performs a random permutation of the nodes
        '''
        # Gets the nodes
        nodes = self.tsp.get_nodes()
        # Makes a deep copy
        initial_solution = copy.deepcopy(nodes)
        # Shuffles them
        random.shuffle(initial_solution)
        return initial_solution



    def swap(self,solution:list, i:int, j:int):
        '''
        Swaps the elements in position i and j in the given solution.
        Dows the following:
        (i) Makes a deep copy of the solution, so that we return a different list
        (ii) Keeps track of the elements in position i and j
        (iii) Swaps them
        '''
        new_solution = copy.deepcopy(solution)
        element_i = new_solution[i]
        element_j = new_solution[j]
        new_solution[i] = element_j
        new_solution[j] = element_i
        return new_solution



    def find_first_improving_solution(self,solution:list):
        current_solution = copy.deepcopy(solution)
        cost_current_solution = self.tsp.get_tour_length(current_solution)
        print("Cost current solution ", cost_current_solution)
        found_improving_solution = False
        for (i, j) in [(a, b) for a in range(len(current_solution)) for b in range(len(current_solution)) if a > b]:
            new_solution = self.swap(current_solution, i, j)
            cost_new_solution = self.tsp.get_tour_length(new_solution)

            if cost_new_solution < cost_current_solution:
                print("Found improving solution with cost", cost_new_solution)
                current_solution = new_solution
                found_improving_solution = True
                # When the first improving solution is found it breaks the loop
                # so that it stops looking for new solutions
                break
        return found_improving_solution, current_solution



    def solve_with_first_improvement(self,initial_solution:list):
        current_solution = copy.deepcopy(initial_solution)
        improving_solution = True
        iteration = 0;
        while improving_solution and (iteration < 500):
            print("Iteration ",iteration)
            improving_solution = False
            iteration += 1
            found_improving_solution, new_solution = self.find_first_improving_solution(current_solution)
            if found_improving_solution:
                improving_solution = found_improving_solution
                current_solution = new_solution
            else:
                print("No improving solution found")
        return current_solution

We create create a local search object

In [7]:
ls = LocalSearch(tsp=tsp)

We create an initial solution

In [11]:
initial_solution = ls.generate_inital_solution()
print(initial_solution)
print("Cost initial solution", tsp.get_tour_length(initial_solution))

[1, 13, 27, 11, 4, 28, 12, 14, 3, 21, 25, 18, 8, 23, 10, 16, 9, 6, 2, 20, 17, 19, 26, 24, 29, 7, 15, 5, 22]
Cost initial solution 6267


We find a local optimum

In [12]:
local_optimum = ls.solve_with_first_improvement(initial_solution)

Iteration  0
Cost current solution  6267
Found improving solution with cost 6160
Iteration  1
Cost current solution  6160
Found improving solution with cost 6053
Iteration  2
Cost current solution  6053
Found improving solution with cost 5995
Iteration  3
Cost current solution  5995
Found improving solution with cost 5899
Iteration  4
Cost current solution  5899
Found improving solution with cost 5792
Iteration  5
Cost current solution  5792
Found improving solution with cost 5704
Iteration  6
Cost current solution  5704
Found improving solution with cost 5686
Iteration  7
Cost current solution  5686
Found improving solution with cost 5617
Iteration  8
Cost current solution  5617
Found improving solution with cost 5448
Iteration  9
Cost current solution  5448
Found improving solution with cost 5441
Iteration  10
Cost current solution  5441
Found improving solution with cost 5321
Iteration  11
Cost current solution  5321
Found improving solution with cost 5311
Iteration  12
Cost current

We compare its cost with the optimal cost

In [13]:
print(local_optimum)
print("Cost local optimum",tsp.get_tour_length(local_optimum))
print(tsp.get_optimal_tour())
print(tsp.get_optimal_tour_length())

[17, 14, 16, 24, 27, 23, 8, 1, 28, 5, 3, 29, 26, 9, 12, 6, 21, 2, 20, 18, 15, 4, 10, 13, 19, 7, 25, 11, 22]
Cost local optimum 2432
[1, 28, 6, 12, 9, 5, 26, 29, 3, 2, 20, 10, 4, 15, 18, 17, 14, 22, 11, 19, 25, 7, 23, 27, 8, 24, 16, 13, 21]
2020
